In [2]:
# default_exp core.store
# export
from ascrum.core.config import Config
from pathlib import Path
import yaml
import os
import time
import getpass

In [7]:
# export
class Store:
    def __init__(self, path):
        self.path = path.resolve()
        self.name = path.stem

    def exists(self):
        return self.path.exists()

    def __getitem__(self, path):
        "Get the content of a story"
        p = self.get_path(path)
        if p.exists():
            with p.open() as f:
                return yaml.load(f, Loader=yaml.FullLoader)
        else:
            return None

    def __setitem__(self, path, story):
        p = self.get_path(path)
        if p.parent.exists():
            with p.open('w') as f:
                yaml.dump(story, f)
            return p.stem
        else:
            raise KeyError(path)

    def __delitem__(self, path):
        p = self.get_path(path)
        if p.exists():
            p.unlink()
            if next(p.parent.iterdir(), None) is None:
                p.parent.unlink()
        else:
            raise KeyError(path)
    
    def mk_path(self, path):
        p = self.path/path
        p.mkdir(parents=True, exist_ok=True)
        return 'Created', 201

    def get_path(self, path):
        return self.path/path

    def touch(self, story_id):
        p = self.get_path(story_id)
        if p.exists():
            p.touch(exist_ok=True)
            return True 
        else:
            return False

    def list(self, path=''):
        p = self.path/path
        l = len(p.as_posix())
        if p.exists():
            return [{
                'name':f.as_posix()[l:], 
                'dir': f.is_dir()
             } for f in sorted(p.rglob('*'), key=os.path.getmtime, 
                    reverse=True)]
        else:
            return None

In [17]:
p = Path('/tmp')
next(p.iterdir())

PosixPath('/tmp/Temp-8b6e03bd-fc85-4966-8299-53f8c66239c6')

In [6]:
p = p[21]
p

PosixPath('web/node_modules/concat-stream/node_modules/readable-stream/lib')

In [7]:
p.parent.as_posix()


'web/node_modules/concat-stream/node_modules/readable-stream'